In [78]:
%matplotlib inline
import pandas as pd
from pandas import Series
import numpy as np
from functools import reduce
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense
from keras.initializers import glorot_normal
from keras.optimizers import Adam
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from numpy import array
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
import sqlite3
import keras
from scipy.special import softmax
from sklearn.metrics import accuracy_score

In [40]:
data = pd.read_csv("data/orig/cleaned_data_notEncoded.csv")

In [41]:
data['label_away'] = data['label_home'].apply(lambda x: -1 * x)

data['home_won'] = data['label_home'].apply(lambda x: 1 if x == 1 else 0)
data['home_loss'] = data['label_home'].apply(lambda x: 1 if x == -1 else 0)
data['home_draw'] = data['label_home'].apply(lambda x: 1 if x == 0 else 0)

data['away_won'] = data['label_away'].apply(lambda x: 1 if x == 1 else 0)
data['away_loss'] = data['label_away'].apply(lambda x: 1 if x == -1 else 0)
data['away_draw'] = data['label_away'].apply(lambda x: 1 if x == 0 else 0)

In [42]:
#match_data = pd.read_csv("data/orig/match_data.csv")
conn = sqlite3.connect('database.sqlite')
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
match_data = pd.read_sql_query("SELECT * FROM Match", conn)

sqlite_sequence
Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


In [43]:
relevant_match_data = match_data[['season', 'match_api_id', 'date', 'home_team_api_id', 'away_team_api_id']]

In [44]:
data_with_season_date = pd.merge(data, relevant_match_data, on='match_api_id')

In [46]:
grouped_home = data_with_season_date.groupby(['season', 'home_team_api_id'])

In [47]:
names = []
groups = []
for name, group in grouped_home:
    names += [name]
    groups += [group]
#     print(name)
#     print(group.shape)

In [49]:
new_groups = []
for i in range(len(groups)):
    df = groups[i]
    name = names[i]
    df['home_win_record'] = df['home_won'].cumsum()
    df['home_loss_record'] = df['home_loss'].cumsum()
    df['home_draw_record'] = df['home_draw'].cumsum()
    df['season'] = name[0]
    df['home_team_api_id'] = name[1]
    new_groups += [df]

In [51]:
df_home_stats = pd.concat(new_groups)

In [52]:
df_home_stats.shape

(20060, 914)

In [53]:
df_home_stats.columns.values

array(['home_player_X1', 'home_player_X2', 'home_player_X3',
       'home_player_X4', 'home_player_X5', 'home_player_X6',
       'home_player_X7', 'home_player_X8', 'home_player_X9',
       'home_player_X10', 'home_player_X11', 'home_player_Y1',
       'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'home_buildUpPlaySpeed',
       'home_buildUpPlayDribbling', 'home_buildUpPlayPassing',
       'home_chanceCreationPassing', 'home_chanceCreationCrossing',
       'home_chanceCreationShooting', 'home_defencePressure',
       'home_defenceAggression', 'home_defenceTeamWidth',
       'home_player_1_overall_rating', 'home_player_1_potential',
       'home_player_1_preferred_foot',
       'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_

In [54]:
grouped_away = df_home_stats.groupby(['season', 'away_team_api_id'])

In [317]:
names = []
groups = []
for name, group in grouped_away:
    names += [name]
    groups += [group]
#     print(name)
#     print(group.shape)
# print(names[4])
# groups[1]

In [56]:
new_groups = []
for i in range(len(groups)):
    df = groups[i]
    name = names[i]
    df['away_win_record'] = df['away_won'].cumsum()
    df['away_loss_record'] = df['away_loss'].cumsum()
    df['away_draw_record'] = df['away_draw'].cumsum()
    df['season'] = name[0]
    df['away_team_api_id'] = name[1]
    new_groups += [df]

In [57]:
df_final = pd.concat(new_groups)

In [58]:
df_final.shape

(20060, 917)

In [59]:
df_final.columns.values

array(['home_player_X1', 'home_player_X2', 'home_player_X3',
       'home_player_X4', 'home_player_X5', 'home_player_X6',
       'home_player_X7', 'home_player_X8', 'home_player_X9',
       'home_player_X10', 'home_player_X11', 'home_player_Y1',
       'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'home_buildUpPlaySpeed',
       'home_buildUpPlayDribbling', 'home_buildUpPlayPassing',
       'home_chanceCreationPassing', 'home_chanceCreationCrossing',
       'home_chanceCreationShooting', 'home_defencePressure',
       'home_defenceAggression', 'home_defenceTeamWidth',
       'home_player_1_overall_rating', 'home_player_1_potential',
       'home_player_1_preferred_foot',
       'home_player_1_attacking_work_rate',
       'home_player_1_defensive_work_rate', 'home_player_1_crossing',
       'home_player_1_finishing', 'home_player_1_

In [60]:
df_final['season'].values

array(['2008/2009', '2008/2009', '2008/2009', ..., '2015/2016',
       '2015/2016', '2015/2016'], dtype=object)

In [61]:
df_final['season'] = pd.factorize(df_final['season'])[0]

In [62]:
df_final = df_final.drop(columns=['label_away', 'home_won', 'home_loss', 'home_draw',
                                  'away_won', 'away_loss', 'away_draw', 'date'])

In [63]:
desired_columns = ['season', 'home_win_record', 'home_loss_record', 'home_draw_record',
                  'away_win_record', 'away_loss_record', 'away_draw_record']

for i in range(1, 12):
    for status in ['home_player', 'away_player']:
        posX_name = status + '_X' + str(i)
        posY_name = status + '_Y' + str(i)
        
        rating_name = status + '_' + str(i) + '_' + 'overall_rating'
        desired_columns.extend([posX_name, posY_name, rating_name])

df_rel = df_final[desired_columns]
df_rel.columns.values

array(['season', 'home_win_record', 'home_loss_record',
       'home_draw_record', 'away_win_record', 'away_loss_record',
       'away_draw_record', 'home_player_X1', 'home_player_Y1',
       'home_player_1_overall_rating', 'away_player_X1', 'away_player_Y1',
       'away_player_1_overall_rating', 'home_player_X2', 'home_player_Y2',
       'home_player_2_overall_rating', 'away_player_X2', 'away_player_Y2',
       'away_player_2_overall_rating', 'home_player_X3', 'home_player_Y3',
       'home_player_3_overall_rating', 'away_player_X3', 'away_player_Y3',
       'away_player_3_overall_rating', 'home_player_X4', 'home_player_Y4',
       'home_player_4_overall_rating', 'away_player_X4', 'away_player_Y4',
       'away_player_4_overall_rating', 'home_player_X5', 'home_player_Y5',
       'home_player_5_overall_rating', 'away_player_X5', 'away_player_Y5',
       'away_player_5_overall_rating', 'home_player_X6', 'home_player_Y6',
       'home_player_6_overall_rating', 'away_player_X6', 'away_pl

In [64]:
df_rel.shape

(20060, 73)

In [65]:
df_label = df_final['label_home']
df_label.shape

(20060,)

In [66]:
X = df_rel.values
y = df_label.values

In [67]:
y = [i + 1 for i in y]
y = array(y)
y = to_categorical(y)
y.shape

(20060, 3)

In [182]:
def recall(y_true, y_pred):
    print(type(y_true))
    print(tf.keras.backend.get_value(y_true))
    recall_vals = recall_score(y_true, y_pred, average=None)
    recall_vals = tf.convert_to_tensor(recall_vals, dtype=tf.float32)
    return recall_vals



In [100]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [199]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
from numpy.random import seed
from tensorflow import set_random_seed
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1).split(X_train, np.argmax(y_train, axis=1))
def createToyModel(input_dim):
    
    xavier_init = glorot_normal(seed=11)
    
    model = Sequential()
    model.add(Dense(units=27, activation='relu', kernel_initializer=xavier_init))
    model.add(Dense(units=9, activation='relu', kernel_initializer=xavier_init))
    model.add(Dense(units=3, activation='relu', kernel_initializer=xavier_init))
    opt = Adam(lr=3e-4)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

seed(5)
set_random_seed(10)
val_loss = []
val_acc = []
for i,f in enumerate(folds):
    fpath = 'kfold_best'+str(i)+'.h5'
    checkpoint = ModelCheckpoint(fpath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    train = f[0]
    val = f[1]

    X_fold_train, y_fold_train = X_train[train], y_train[train]
    X_fold_val, y_fold_val = X_train[val], y_train[val]
    
    model = createToyModel(X_train.shape[1])
    model.fit(x=X_fold_train, y=y_fold_train, batch_size=8, shuffle=True, epochs=30, 
    validation_data=(X_fold_val, y_fold_val), callbacks=callbacks_list)
    
    model.load_weights(fpath)
    loss, acc = model.evaluate(X_fold_val, y_fold_val)
    val_loss.append(loss)
    val_acc.append(acc)

Train on 14442 samples, validate on 3612 samples
Epoch 1/30
14442/14442 [==============================] - 11s 786us/step - loss: 1.0643 - acc: 0.4580 - val_loss: 1.0523 - val_acc: 0.4599
Epoch 2/30
14442/14442 [==============================] - 8s 540us/step - loss: 1.0533 - acc: 0.4621 - val_loss: 1.0447 - val_acc: 0.4612
Epoch 3/30
14442/14442 [==============================] - 8s 583us/step - loss: 1.0300 - acc: 0.4819 - val_loss: 1.0176 - val_acc: 0.5089
Epoch 4/30
14442/14442 [==============================] - 8s 561us/step - loss: 1.0322 - acc: 0.4883 - val_loss: 1.0392 - val_acc: 0.4579
Epoch 5/30
14442/14442 [==============================] - 9s 623us/step - loss: 1.0173 - acc: 0.4803 - val_loss: 1.0311 - val_acc: 0.5055
Epoch 6/30
14442/14442 [==============================] - 8s 575us/step - loss: 1.0295 - acc: 0.5110 - val_loss: 0.9989 - val_acc: 0.4925
Epoch 7/30
14442/14442 [==============================] - 8s 569us/step - loss: 1.0218 - acc: 0.4708 - val_loss: 1.0123 - 

Epoch 29/30
14442/14442 [==============================] - 8s 557us/step - loss: 0.9301 - acc: 0.5839 - val_loss: 0.8979 - val_acc: 0.5944
Epoch 30/30
3612/3612 [==============================] - 0s 94us/step
Train on 14443 samples, validate on 3611 samples
Epoch 1/30
14443/14443 [==============================] - 12s 837us/step - loss: 1.0630 - acc: 0.4564 - val_loss: 1.0519 - val_acc: 0.4608
Epoch 2/30
14443/14443 [==============================] - 8s 583us/step - loss: 1.0459 - acc: 0.4643 - val_loss: 1.0375 - val_acc: 0.4580
Epoch 3/30
14443/14443 [==============================] - 8s 588us/step - loss: 1.0293 - acc: 0.4811 - val_loss: 1.0153 - val_acc: 0.4675
Epoch 4/30
14443/14443 [==============================] - 9s 593us/step - loss: 1.0325 - acc: 0.4820 - val_loss: 1.0367 - val_acc: 0.4592
Epoch 5/30
14443/14443 [==============================] - 8s 586us/step - loss: 1.0082 - acc: 0.4801 - val_loss: 0.9445 - val_acc: 0.5708
Epoch 6/30
14443/14443 [===========================

14444/14444 [==============================] - 9s 589us/step - loss: 0.8832 - acc: 0.5909 - val_loss: 0.8923 - val_acc: 0.6025
Epoch 27/30
14444/14444 [==============================] - 8s 588us/step - loss: 0.8701 - acc: 0.6061 - val_loss: 0.8864 - val_acc: 0.5903
Epoch 28/30
14444/14444 [==============================] - 9s 607us/step - loss: 0.8562 - acc: 0.6116 - val_loss: 0.8560 - val_acc: 0.6100
Epoch 29/30
14444/14444 [==============================] - 9s 600us/step - loss: 0.8538 - acc: 0.6151 - val_loss: 0.8458 - val_acc: 0.6155
Epoch 30/30
3610/3610 [==============================] - 0s 113us/step
Train on 14445 samples, validate on 3609 samples
Epoch 1/30
14445/14445 [==============================] - 13s 908us/step - loss: 1.0652 - acc: 0.4559 - val_loss: 1.0524 - val_acc: 0.4580
Epoch 2/30
14445/14445 [==============================] - 9s 623us/step - loss: 1.0449 - acc: 0.4674 - val_loss: 1.0290 - val_acc: 0.5004
Epoch 3/30
14445/14445 [==============================] - 9

In [207]:
print('Val losses:', val_loss)
print('Val accs:', val_acc)
print('Avg. val acc', np.mean(val_acc))

Val losses: [0.9327828295999191, 0.9382638246118032, 0.9329649090964983, 0.853027103416147, 0.945133680731149]
Val accs: [0.5991140642963506, 0.6052048725807258, 0.5984491832498631, 0.6168975070572956, 0.61928512043291]
Avg. val acc 0.607790149523429


In [208]:
model.evaluate(X_test, y_test)

2006/2006 [==============================] - 0s 135us/step


[0.9420243557523992, 0.6026919244650235]

In [209]:
y_pred = model.predict(x=X_test)

p_score_avg, r_score_avg, f1_score_avg, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(y_pred, axis=1).astype(int), 3), average='weighted')
print("Averages")
print(p_score_avg)
print(r_score_avg)
print(f1_score_avg)


p_scores, r_scores, f1_scores, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(y_pred, axis=1).astype(int), 3), average=None)
print("Per Class")
print(p_scores)
print(r_scores)
print(f1_scores)

Averages
0.6039009226615001
0.6026919242273181
0.5905365408287573
Per Class
[0.63116883 0.5704388  0.60521886]
[0.41680961 0.48336595 0.78837719]
[0.50206612 0.52330508 0.6847619 ]


In [213]:
def createToyModel(input_dim):
    
    xavier_init = glorot_normal(seed=11)
    
    model = Sequential()
    model.add(Dense(units=27, activation='relu', kernel_initializer=xavier_init))
    model.add(Dense(units=9, activation='relu', kernel_initializer=xavier_init))
    model.add(Dense(units=3, activation='relu', kernel_initializer=xavier_init))
    opt = Adam(lr=3e-4)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [228]:
train_loss = []
train_acc = []
predicted = []
model = createToyModel(X_train.shape[1])
model.fit(x=X_fold_train, y=y_fold_train, batch_size=8, shuffle=True, epochs=1, 
    validation_data=(X_fold_val, y_fold_val), callbacks=callbacks_list)

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1).split(X_train, np.argmax(y_train, axis=1))
for i,f in enumerate(folds):
    fpath = 'kfold_best'+str(i)+'.h5'
    train = f[0]
    val = f[1]

    X_fold_train, y_fold_train = X_train[train], y_train[train]
    X_fold_val, y_fold_val = X_train[val], y_train[val]

    
    
    model.load_weights(fpath)
    loss, acc = model.evaluate(X_fold_train, y_fold_train)
    y_pred = model.predict(x=X_test)
    predicted.append(y_pred)
    train_loss.append(loss)
    train_acc.append(acc)

Train on 14445 samples, validate on 3609 samples
Epoch 1/1
14445/14445 [==============================] - 2s 111us/step


In [229]:
print('Val losses:', val_loss)
print('Val accs:', val_acc)
print('Avg. val acc', np.mean(val_acc))

Val losses: [0.9327828295999191, 0.9382638246118032, 0.9329649090964983, 0.853027103416147, 0.945133680731149]
Val accs: [0.5991140642963506, 0.6052048725807258, 0.5984491832498631, 0.6168975070572956, 0.61928512043291]
Avg. val acc 0.607790149523429


In [230]:
print(np.mean(np.array(val_acc) - np.array(train_acc)))

-0.0010080243112336573


In [318]:
df = pd.DataFrame(index=pd.Index(['Model ' + str(i) for i in range(5)] + ['Ensemble averaging']), columns=['precision', 'recall', 'f1', 'accuracy']).fillna(0)
for i, p in enumerate(predicted):
    p_score, r_score, f1_score, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(p, axis=1).astype(int), 3), average='weighted')
    acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(p, axis=1))
    df.loc['Model ' +str(i), 'precision'] = p_score
    df.loc['Model ' +str(i), 'recall'] = r_score
    df.loc['Model ' +str(i), 'f1'] = f1_score
    df.loc['Model ' +str(i), 'accuracy'] = acc
df

,precision,recall,f1,accuracy
Model 0,0.604615,0.593220,0.586863,0.593220
Model 1,0.608607,0.611665,0.605057,0.611665
Model 2,0.612161,0.609172,0.597258,0.609172
Model 3,0.623026,0.613659,0.608883,0.613659
Model 4,0.603901,0.602692,0.590537,0.602692
Ensemble averaging,0.000000,0.000000,0.000000,0.000000


In [258]:
# normalize predictions for all models
predicted_arr = np.array(predicted)
for i in range(5):
    for j in range(predicted_arr.shape[1]):
        predicted_arr[i, j, :] /= np.linalg.norm(predicted_arr[i, j, :] )

In [260]:
predicted_mean = np.mean(predicted_arr, axis=0)

In [319]:
# plt.figure()
# plt.hist(np.argmax(predicted_mean, axis=1), color='r', alpha=0.5, label='pred')
# plt.hist(np.argmax(y_test, axis=1), color='yellow', alpha=0.3, label='true')
p_score, r_score, f1_score, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(predicted_mean, axis=1).astype(int), 3), average='weighted')
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predicted_mean, axis=1))
df.loc['Ensemble averaging', 'precision'] = p_score
df.loc['Ensemble averaging', 'recall'] = r_score
df.loc['Ensemble averaging', 'f1'] = f1_score
df.loc['Ensemble averaging', 'accuracy'] = acc

df.to_csv('prediction_scores.csv')
df

,precision,recall,f1,accuracy
Model 0,0.604615,0.593220,0.586863,0.593220
Model 1,0.608607,0.611665,0.605057,0.611665
Model 2,0.612161,0.609172,0.597258,0.609172
Model 3,0.623026,0.613659,0.608883,0.613659
Model 4,0.603901,0.602692,0.590537,0.602692
Ensemble averaging,0.632584,0.633599,0.628527,0.633599
